# EDA - Data Preprocessing

In [7]:
# import the necessary libraries 
import duckdb
import pandas as pd

In [10]:
# Load csv to DuckDB

# Connect to a DuckDB database file (this will create 'my_database.duckdb' if it doesn't exist)
conn = duckdb.connect('../Instance/water.duckdb')

conn.execute(
    """
    CREATE TABLE IF NOT EXISTS water_data AS 
    SELECT * FROM read_csv_auto('../Data/global_water_consumption.csv')
    """
)

In [11]:
# query the data to get the first 5 rows 
conn.sql("SELECT * FROM water_data LIMIT 5").fetchdf()


,Country,Year,Total Water Consumption (Billion Cubic Meters),Per Capita Water Use (Liters per Day),Water Scarcity Level,Agricultural Water Use (%),Industrial Water Use (%),Household Water Use (%),Rainfall Impact (Annual Precipitation in mm),Groundwater Depletion Rate (%)
0,Indonesia,2022,895.15,489.73,Low,20.78,13.75,34.99,1075.28,3.10
1,Indonesia,2024,502.89,311.95,High,48.51,8.44,32.88,2630.69,1.78
2,Spain,2000,843.39,440.09,Medium,25.16,31.70,34.62,2860.62,4.13
3,Canada,2021,803.34,478.98,High,45.74,6.13,18.99,1725.50,0.61
4,Brazil,2022,416.40,353.91,High,26.58,7.95,31.11,988.44,0.80


The names of the columns are quite long and hence needs to be shortened for easy usage. 

In [12]:
# Rename columns for easier handling
query = """
    SELECT 
        Country AS country,
        Year AS year,
        "Total Water Consumption (Billion Cubic Meters)" AS total_water_consumption_bcm,
        "Per Capita Water Use (Liters per Day)" AS per_capita_water_use_lpd,
        "Water Scarcity Level" AS water_scarcity_level,
        "Agricultural Water Use (%)" AS agricultural_water_use_pct,
        "Industrial Water Use (%)" AS industrial_water_use_pct,
        "Household Water Use (%)" AS household_water_use_pct,
        "Rainfall Impact (Annual Precipitation in mm)" AS rainfall_impact_mm,
        "Groundwater Depletion Rate (%)" AS groundwater_depletion_rate_pct
    FROM water_data
    ORDER BY Year
"""

water_df = conn.sql(query=query).fetchdf()
water_df.head()

,country,year,total_water_consumption_bcm,per_capita_water_use_lpd,water_scarcity_level,agricultural_water_use_pct,industrial_water_use_pct,household_water_use_pct,rainfall_impact_mm,groundwater_depletion_rate_pct
0,Spain,2000,843.39,440.09,Medium,25.16,31.70,34.62,2860.62,4.13
1,Indonesia,2000,485.84,419.38,Medium,31.49,25.18,13.70,1883.89,1.11
2,France,2000,613.43,417.64,Medium,77.75,13.71,23.43,2530.80,4.90
3,Spain,2000,61.53,362.74,Medium,33.22,19.40,11.23,1590.81,4.91
4,South Africa,2000,818.07,142.17,Low,64.77,18.07,36.31,291.64,4.96


In [47]:
conn.sql(
    """
    SELECT year , COUNT(*) AS Observations
    FROM water_data 
    GROUP BY year
    ORDER BY year
    """
)

┌───────┬──────────────┐
│ Year  │ Observations │
│ int64 │    int64     │
├───────┼──────────────┤
│  2000 │          172 │
│  2001 │          199 │
│  2002 │          212 │
│  2003 │          202 │
│  2004 │          190 │
│  2005 │          196 │
│  2006 │          169 │
│  2007 │          217 │
│  2008 │          191 │
│  2009 │          185 │
│    ·  │           ·  │
│    ·  │           ·  │
│    ·  │           ·  │
│  2015 │          175 │
│  2016 │          240 │
│  2017 │          191 │
│  2018 │          194 │
│  2019 │          197 │
│  2020 │          209 │
│  2021 │          225 │
│  2022 │          219 │
│  2023 │          195 │
│  2024 │          204 │
├───────┴──────────────┤
│  25 rows (20 shown)  │
└──────────────────────┘

In [46]:
conn.sql(
    """
    SELECT country , COUNT(*) as Observations
    FROM water_data 
    GROUP BY country
    ORDER BY country
    """
)

┌──────────────┬──────────────┐
│   Country    │ Observations │
│   varchar    │    int64     │
├──────────────┼──────────────┤
│ Argentina    │          260 │
│ Australia    │          268 │
│ Brazil       │          241 │
│ Canada       │          250 │
│ China        │          252 │
│ France       │          248 │
│ Germany      │          237 │
│ India        │          241 │
│ Indonesia    │          240 │
│ Italy        │          252 │
│ Japan        │          211 │
│ Mexico       │          230 │
│ Russia       │          266 │
│ Saudi Arabia │          276 │
│ South Africa │          273 │
│ South Korea  │          245 │
│ Spain        │          241 │
│ Turkey       │          269 │
│ UK           │          237 │
│ USA          │          263 │
├──────────────┴──────────────┤
│ 20 rows           2 columns │
└─────────────────────────────┘

So, each year has a different set of observations. Also, the number of observations for each country is not the same. 

In [45]:
conn.sql(
    """
    SELECT country , year,  COUNT(*) AS Observations
    FROM water_data 
    GROUP BY year, country
    ORDER BY country, year
    """
)

┌───────────┬───────┬──────────────┐
│  Country  │ Year  │ Observations │
│  varchar  │ int64 │    int64     │
├───────────┼───────┼──────────────┤
│ Argentina │  2000 │            7 │
│ Argentina │  2001 │           10 │
│ Argentina │  2002 │           13 │
│ Argentina │  2003 │           15 │
│ Argentina │  2004 │            5 │
│ Argentina │  2005 │           10 │
│ Argentina │  2006 │            6 │
│ Argentina │  2007 │            4 │
│ Argentina │  2008 │           10 │
│ Argentina │  2009 │            6 │
│  ·        │    ·  │            · │
│  ·        │    ·  │            · │
│  ·        │    ·  │            · │
│ USA       │  2015 │           14 │
│ USA       │  2016 │           10 │
│ USA       │  2017 │           10 │
│ USA       │  2018 │            6 │
│ USA       │  2019 │           16 │
│ USA       │  2020 │           10 │
│ USA       │  2021 │           10 │
│ USA       │  2022 │           10 │
│ USA       │  2023 │            8 │
│ USA       │  2024 │           10 │
├

Note, the number of observations for each country for a particular year is also not the same.

In [44]:
# the distinct names of the countries 

conn.sql(
    """
    SELECT DISTINCT(country) FROM water_data
    """
)

┌──────────────┐
│   Country    │
│   varchar    │
├──────────────┤
│ Indonesia    │
│ France       │
│ Italy        │
│ Russia       │
│ South Korea  │
│ India        │
│ UK           │
│ Australia    │
│ Saudi Arabia │
│ Spain        │
│ China        │
│ Brazil       │
│ Germany      │
│ Turkey       │
│ South Africa │
│ Canada       │
│ Argentina    │
│ Mexico       │
│ USA          │
│ Japan        │
├──────────────┤
│   20 rows    │
└──────────────┘

In [49]:
# the distinct years 

conn.sql(
    """
    SELECT DISTINCT(year) FROM water_data ORDER BY year
    """
)

┌────────────┐
│    Year    │
│   int64    │
├────────────┤
│       2000 │
│       2001 │
│       2002 │
│       2003 │
│       2004 │
│       2005 │
│       2006 │
│       2007 │
│       2008 │
│       2009 │
│         ·  │
│         ·  │
│         ·  │
│       2015 │
│       2016 │
│       2017 │
│       2018 │
│       2019 │
│       2020 │
│       2021 │
│       2022 │
│       2023 │
│       2024 │
├────────────┤
│  25 rows   │
│ (20 shown) │
└────────────┘

In [50]:
# get info
water_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 10 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   country                         5000 non-null   object 
 1   year                            5000 non-null   int64  
 2   total_water_consumption_bcm     5000 non-null   float64
 3   per_capita_water_use_lpd        5000 non-null   float64
 4   water_scarcity_level            5000 non-null   object 
 5   agricultural_water_use_pct      5000 non-null   float64
 6   industrial_water_use_pct        5000 non-null   float64
 7   household_water_use_pct         5000 non-null   float64
 8   rainfall_impact_mm              5000 non-null   float64
 9   groundwater_depletion_rate_pct  5000 non-null   float64
dtypes: float64(7), int64(1), object(2)
memory usage: 390.8+ KB


There is no null values inside the dataset.

In [51]:
# let us save the renamed dataset 
conn.sql(query=query).write_csv('../Data/renamed_global_water_consumption.csv')

In [52]:
conn.close()